In [ ]:
!pip install pytube pydub youtube_transcript_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install -U openai-whisper jiwer datasets[audio]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pytube
from pydub import AudioSegment
from youtube_transcript_api import YouTubeTranscriptApi
import json
import os
import whisper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/cs_172b_project/

/content/drive/My Drive/cs_172b_project


In [ ]:
added_videos = ["https://www.youtube.com/watch?v=xBlqqMudb8k", "https://www.youtube.com/watch?v=84jI3CLIRSI",
                "https://www.youtube.com/watch?v=2sBgPkjj5uw", "https://www.youtube.com/watch?v=KRbWAQ0TKXQ&t=1s",
                "https://www.youtube.com/watch?v=XsHAlw1mGtk", "https://www.youtube.com/watch?v=qMvKAHk2J7s"]
video_url = 'https://www.youtube.com/watch?v=xBlqqMudb8k'
transcript_file = './yt_transcripts/Deep Learning in Biomdedical Imaging.json'
output_dir = './learning_in_the_machine_audio_clips'

In [ ]:
import re

def get_youtube_transcript(url):
    # Extract video_id from URL
    video_id_match = re.search(r'(?<=v=)[^&#]+', url)
    video_id_match = video_id_match or re.search(r'(?<=be/)[^&#]+', url)
    video_id = (video_id_match.group(0) if video_id_match
                 else None)
    if not video_id:
        raise ValueError('Incorrect YouTube URL')

    # Fetch the transcript
    transcript = YouTubeTranscriptApi.get_transcript(video_id)

    return transcript

In [ ]:
def get_segments(transcript, target_duration=15.0, min_word_len = 4):
    segments = []
    current_segment = []
    current_duration = 0.0

    def add_segment():
        nonlocal current_segment
        nonlocal current_duration
        nonlocal target_duration

        if len(current_segment) > 0 and current_duration > 0:
            if current_duration > target_duration:
                segments.append(current_segment)
                current_segment = []
                current_duration = 0.0
            elif current_duration < target_duration:
                pass
            else:
                current_segment = []
                current_duration = 0.0

    for i in range(len(transcript)):
        # Calculate the end time of the current clip
        end_time = transcript[i]['start'] + transcript[i]['duration']

        # Check if the next clip starts before the current one ends (overlap)
        if i < len(transcript) - 1 and transcript[i+1]['start'] < end_time:
            overlap = end_time - transcript[i+1]['start']
            # If the overlap is more than half of the next clip, skip it
            if overlap / transcript[i+1]['duration'] > 0.75:
                add_segment()
                continue

        if len(transcript[i]['text'].split()) < min_word_len:
            add_segment()
            continue


        # Add the current clip to the current segment and update the duration
        current_segment.append(transcript[i])
        current_duration += transcript[i]['duration']

        # If the current segment is long enough, add it to the list of segments
        add_segment()

    # If there's a segment that's less than target_duration but not empty, add it to the segments
    add_segment()

    return segments


In [ ]:
def merge_segments(segments):
    merged_segments = []

    for segment in segments:
        merged_text = ' '.join([clip['text'] for clip in segment])
        start_time = segment[0]['start']
        end_time = segment[-1]['start'] + segment[-1]['duration']

        merged_segments.append({
            'text': merged_text,
            'start': start_time,
            'end': end_time,
        })

    return merged_segments

In [ ]:
def video_already_processed(output_path, video_title):
    for filename in os.listdir(output_path):
        if video_title in filename:
            return True
    return False

In [ ]:
from pytube import YouTube

def download_audio_clips(url, output_path):
    yt = YouTube(url)
    video_title = yt.title  # get video title

    # Check if the video has already been processed
    if video_already_processed(output_path, video_title):
        print(f"Video '{video_title}' has already been processed.")
        return

    transcripts = get_youtube_transcript(url)
    segments = merge_segments(get_segments(transcripts))

    video = yt.streams.filter(only_audio=True).first()
    filename = video.download(output_path)

    # Convert video to .wav format
    audio = AudioSegment.from_file(filename, "mp4")
    base_filename, _ = os.path.splitext(filename)
    audio_filename = f"{base_filename}.wav"
    audio.export(audio_filename, format="wav")

    audio = AudioSegment.from_wav(audio_filename)

    for i, segment in enumerate(segments):
        start_ms = segment['start'] * 1000  # convert to milliseconds
        end_ms = segment['end'] * 1000
        clip = audio[start_ms:end_ms]
        clip_filename = f"{output_path}/{video_title}_clip_{i+1}_{segment['start']}_{segment['end']}.wav"
        clip.export(clip_filename, format="wav")

    # Remove the original files
    os.remove(filename)
    os.remove(audio_filename)

In [ ]:
import csv


def transcribe_and_save(output_path, model):
    metadata_file = os.path.join(output_path, 'metadata.csv')
    file_exists = os.path.isfile(metadata_file)
    with open(metadata_file, 'a', newline='') as file:
        csvwriter = csv.writer(file, delimiter=',')
        if not file_exists:
            csvwriter.writerow(['file_name', 'transcription'])
        for chunk in os.listdir(output_path):
            if chunk.endswith('.wav'):
                fn = os.path.join(output_path, chunk)
                result = model.transcribe(fn)
                csvwriter.writerow([chunk, result['text']])

In [ ]:
download_audio_clips(video_url, './full_pierre_dataset')

In [ ]:
model = whisper.load_model('base')
model.device

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.4MiB/s]


device(type='cuda', index=0)

In [ ]:
transcribe_and_save('./full_pierre_dataset', model)

In [ ]:
from datasets import load_dataset, Audio
dataset = load_dataset("audiofolder", data_dir="full_pierre_dataset/")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

Resolving data files:   0%|          | 0/1023 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/default-b765a0972173e081/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1022
    })
})

In [ ]:
# test adding second video
url = 'https://www.youtube.com/watch?v=qMvKAHk2J7s'
download_audio_clips(url, './full_pierre_dataset')
transcribe_and_save('./full_pierre_dataset', model)

In [ ]:
dataset = load_dataset("audiofolder", data_dir="./full_pierre_dataset")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
dataset

Resolving data files:   0%|          | 0/2240 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/default-46cb6970984ad9fb/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2239
    })
})

In [ ]:
def extract_audio_clips(video_url, transcript_file, output_dir):
    # Load transcript from JSON file
    with open(transcript_file, 'r') as f:
        transcript = json.load(f)

    # Download yt video
    youtube = pytube.YouTube(video_url)
    video = youtube.streams.first()
    video_filename = video.download(output_path=output_dir)

    # Load video as an AudioSegment
    video_audio = AudioSegment.from_file(video_filename)

    # Extract audio clips based on the transcript
    for entry in transcript:
        text = entry['text']
        start_time = entry['start']
        duration = entry['duration']

        # Calculate start and end timestamps in milliseconds
        start_time_ms = int(start_time * 1000)
        end_time_ms = int((start_time + duration) * 1000)

        # Extract the audio clip
        audio_clip = video_audio[start_time_ms:end_time_ms]

        # Generate the output filename
        output_filename = f"{output_dir}/{start_time}_{start_time+duration}.wav"

        # Export the audio clip
        audio_clip.export(output_filename, format='wav')

    # Clean up temporary video file
    os.remove(video_filename)

In [ ]:
extract_audio_clips(video_url, transcript_file, output_dir)